In [ ]:
#default_exp data.source

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.pipeline import *
from local.notebook.showdoc import show_doc

In [ ]:
#hide
torch.cuda.set_device(int(os.environ.get('DEFAULT_GPU') or 0))

# Data source
> Base container for all the items

## Convenience functions

In [ ]:
#export core
def all_union(sets):
    "Set of union of all `sets` (each `setified` if needed)"
    return set().union(*(map(setify,sets)))

In [ ]:
sets = [[1,2],[2,3]]
test_eq(all_union(sets), {1,2,3})

In [ ]:
#export core
def all_disjoint(sets):
    "`True` iif no element appears in more than one item of `sets`"
    return sum(map(len,sets))==len(all_union(sets))

In [ ]:
assert not all_disjoint(sets)
assert all_disjoint([[1,2],[3,4]])
assert all_disjoint([[1,2],[]])
assert all_disjoint([[1,2]])
assert all_disjoint([])

## DataSource -

In [ ]:
# export
class _FiltTfmdList(TfmdList):
    "Like `TfmdList` but with filters and train/valid attribute, for proper setup"
    def __init__(self, items, tfms, filts, filt_idx, do_setup=True):
        self.filts,self.filt_idx = filts,filt_idx
        super().__init__(items, tfms, do_setup=do_setup)

    def __getitem__(self, i, filt=None): #Has to accept filt because of DataSource.__getitem__ and super behavior
        return super().__getitem__(i,self.filt_idx[i] if filt is None else filt)
    
    @property
    def n_subsets(self): return len(self.filts)
    def len(self,filt): return len(self.filts[filt])
    def subset(self, i): return DsrcSubset(self, i)
    def subsets(self): return map(self.subset, range(self.n_subsets))
    
_FiltTfmdList.train,_FiltTfmdList.valid = add_props(lambda i,x: x.subset(i), 2)

In [ ]:
# export
@docs
class DataSource(TfmdDS):
    "Applies a `tfm` to filtered subsets of `items`"
    def __init__(self, items, tfms=None, tuple_tfms=None, filts=None, do_setup=True):
        if filts is None: filts = [range_of(items)]
        self.filts = L(mask2idxs(filt) for filt in filts)
        # Create map from item id to filter id
        assert all_disjoint(self.filts)
        self.filt_idx = L([None]*len(items))
        for i,f in enumerate(self.filts): self.filt_idx[f] = i
        if tfms is None: tfms = [None]
        self.items = items
        self.tfmd_its = [_FiltTfmdList(items, t, self.filts, self.filt_idx, do_setup=do_setup) for t in tfms]
        self.tfms = [it.tfms for it in self.tfmd_its]
        self.tuple_tfms = Pipeline(tuple_tfms, t=[it.tfms.final_t for it in self.tfmd_its])
        if do_setup: self.setup()

    @property
    def n_subsets(self): return len(self.filts)
    def len(self,filt): return len(self.filts[filt])
    def subset(self, i): return DsrcSubset(self, i)
    def subsets(self): return map(self.subset, range(self.n_subsets))
    def __repr__(self): return '\n'.join(map(str,self.subsets())) + f'\ntfm - {self.tfms}'
    def __getitem__(self, i): return super().__getitem__(i,self.filt_idx[i])
        
    _docs = dict(len="`len` of subset `filt`",
                 __getitem__="Transformed item(s) at `i`, using the appropriate filter",
                 setup="Transform setup",
                 subset="Filtered `DsrcSubset` `i`",
                 subsets="Iterator for all subsets")

DataSource.train,DataSource.valid = add_props(lambda i,x: x.subset(i), 2)

A `DataSource` provides filtering and transformation capabilities to a list of items. Although it has all the attributes of `PipedList` (since it's a subclass) they are mainly used internally; you will generally want to instead access its `subset`s.

If you don't pass any filters or transforms, it simply provides a single subset (of type `DsrcSubset`) with the same behavior as a `L`.

In [ ]:
inp = [0,1,2,3,4]
dsrc = DataSource(inp)

test_eq(dsrc,inp)            # No filters, so equal to input items
test_eq(dsrc.n_subsets, 1)
test_ne(dsrc, [0,1,2,3,5])
test_eq(dsrc[2], 2)          # Retrieve one item (subset 0 is the default)
test_eq(dsrc[1,2], [1,2])    # Retrieve two items by index
mask = [True,False,False,True,False]
test_eq(dsrc[mask], [0,3])   # Retrieve two items by mask

In [ ]:
#export
@docs
class DsrcSubset():
    "A filtered subset of a `DataSource`"
    def __init__(self, dsrc, filt): 
        self.dsrc,self.filt,self.filts = dsrc,filt,dsrc.filts[filt]
        self.tfms,self.tuple_tfms = dsrc.tfms,getattr(dsrc, 'tuple_tfms', None)
    def __getitem__(self,i):             return self.dsrc[self.filts[i]]
    def decode(self, o, **kwargs):       return self.dsrc.decode(o, filt=self.filt, **kwargs)
    def decode_batch(self, b, **kwargs): return self.dsrc.decode_batch(b, filt=self.filt, **kwargs)
    def decode_at(self, i, **kwargs):    return self.decode(self[i], **kwargs)
    def show     (self, o, **kwargs):    return self.dsrc.show(o, filt=self.filt, **kwargs)
    def show_at  (self, i, **kwargs):    return self.dsrc.show(self[i], filt=self.filt, **kwargs)
    def __len__(self):  return len(self.filts)
    def __eq__(self,b): return all_equal(b,self)
    def __repr__(self): return coll_repr(self)
    
    _docs = dict(decode="Transform decode",
                 show="Transform show",
                 decode_batch="Transform decode batch",
                 __getitem__="Encoded item(s) at `i`",
                 decode_at="Decoded item at `i`",
                 show_at="Show decoded item at `i`")

Passing `filts` to the `DataSource` constructor allows you to create multiple subsets, each of type `DsrcSubset`.

In [ ]:
dsrc.items

[0, 1, 2, 3, 4]

In [ ]:
# filts can be indices
dsrc = DataSource(range(5), filts=[tensor([0,2]), [1,3,4]])

test_eq(dsrc.n_subsets, 2)
test_eq(dsrc.subset(0), [0,2])
test_eq(dsrc.train, [0,2])       # Subset 0 is aliased to `train`
test_eq(dsrc.subset(1), [1,3,4])
test_eq(dsrc.valid, [1,3,4])     # Subset 1 is aliased to `valid`
test_eq(dsrc.valid[2], 4)
assert '[1,3,4]' in str(dsrc) and '[0,2]' in str(dsrc)
dsrc

(#2) [0,2]
(#3) [1,3,4]
tfm - [Pipeline over []]

In [ ]:
# filts can be boolean masks (they don't have to cover all items, but must be disjoint)
filts = [[False,True,True,False,True], [True,False,False,False,False]]
dsrc = DataSource(range(5), filts=filts)

test_eq(dsrc.train, [1,2,4])
test_eq(dsrc.valid, [0])

Pass `tfms` to have transformations applied before returning items.

In [ ]:
# apply transforms to all items
tfm = [[lambda x: x*2,lambda x: x+1]]
filts = [[1,2],[0,3,4]]
dsrc = DataSource(range(5), tfm, filts=filts)
test_eq(dsrc.train,[3,5])
test_eq(dsrc.valid,[1,7,9])
test_eq(dsrc.train[False,True], [5])

The subset index is also passed to your transform, so if it is an instance of `Transform` it will only be applied if the filt idx matches.

In [ ]:
# only transform subset 1
class _Tfm(Transform):
    filt=1
    def encodes(self, x)->String: return x*2
    def decodes(self, x): return x//2

In [ ]:
dsrc = DataSource(range(5), [_Tfm()], filts=[[1,2],[0,3,4]])
test_eq(dsrc.train,[1,2])
test_eq(dsrc.valid,[0,6,8])
test_eq(dsrc.train[False,True], [2])
test_eq(dsrc[1,3], [1,6]) #Items are grabbed properly even if they correspond to different filters
dsrc

(#2) [1,2]
(#3) [0,6,8]
tfm - [Pipeline over [<__main__._Tfm object at 0x7f98cea4fb00>]]

In [ ]:
#hide
#Test setup works with train attribute
class _Cat(Transform):
    order,state_args = 1,'vocab'
    def __init__(self, subset_idx=None): self.subset_idx = subset_idx
    def encodes(self, o): return self.o2i[o]
    def decodes(self, o): return self.vocab[o]
    def setup(self, items): 
        items = items.train
        self.vocab,self.o2i = uniqueify(items, sort=True, bidir=True)

def _lbl(o) -> String: return o.split('_')[0]

test_fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tcat = _Cat()
dsrc = DataSource(test_fns, [[tcat,_lbl]], filts=[[1,2,4], [0,3]])
test_eq(tcat.vocab, ['cat','dog'])
test_eq(dsrc.train, [0,0,1])
test_eq(dsrc.valid, [1,0])
test_stdout(lambda: dsrc.train.show_at(0), "cat")

In [ ]:
dsrc.vocab

['cat', 'dog']

In [ ]:
#hide
#Test DataSource pickles
dsrc1 = pickle.loads(pickle.dumps(dsrc))
test_eq(dsrc.train, dsrc1.train)
test_eq(dsrc.valid, dsrc1.valid)
test_eq(dsrc1.vocab, ['cat','dog'])

In [ ]:
dsrc = DataSource(range(5), [_Tfm(),mk_string], filts=[[1,2],[0,3,4]])
test_eq(dsrc.train,[[1,1],[2,2]])
test_eq(dsrc.valid,[[0,0],[6,3],[8,4]])

### `DataSource` Methods

You won't need to use many methods of `DataSource`, since normally you'll be accessing subsets, and therefore will be using `DsrcSubset` methods. However there are a few `DataSource` methods that may be useful:

In [ ]:
show_doc(DataSource.len)

<h4 id="<code>DataSource.len</code>" class="doc_header"><code>DataSource.len</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DataSource.len</code>(**`filt`**)

`len` of subset `filt`

In [ ]:
[dsrc.len(i) for i in range(2)]

[2, 3]

In [ ]:
show_doc(DataSource.subset)

<h4 id="<code>DataSource.subset</code>" class="doc_header"><code>DataSource.subset</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DataSource.subset</code>(**`i`**)

Filtered [`DsrcSubset`](/data.source.html#DsrcSubset) `i`

Subset 0 is aliased to the `train` property, and subset 1 is aliased to the `valid` property.

In [ ]:
dsrc.subset(1)

(#3) [(0, 0),(6, 3),(8, 4)]

In [ ]:
show_doc(DataSource.subsets)

<h4 id="<code>DataSource.subsets</code>" class="doc_header"><code>DataSource.subsets</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DataSource.subsets</code>()

Iterator for all subsets

In [ ]:
for i,s in enumerate(dsrc.subsets()): print(f"Subset {i}: {s}")

Subset 0: (#2) [(1, 1),(2, 2)]
Subset 1: (#3) [(0, 0),(6, 3),(8, 4)]


### `DsrcSubset` Methods

In [ ]:
show_doc(DsrcSubset.__getitem__)

<h4 id="<code>DsrcSubset.__getitem__</code>" class="doc_header"><code>DsrcSubset.__getitem__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DsrcSubset.__getitem__</code>(**`i`**)

Encoded item(s) at `i`

In [ ]:
test_eq(dsrc.valid[1], [6,3])
test_eq(dsrc.train[1], [2,2])

In [ ]:
show_doc(DsrcSubset.show_at)

<h4 id="<code>DsrcSubset.show_at</code>" class="doc_header"><code>DsrcSubset.show_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DsrcSubset.show_at</code>(**`i`**, **\*\*`kwargs`**)

Show decoded item at `i`

In [ ]:
test_stdout(lambda:dsrc.valid.show_at(1), '6\n3')
test_stdout(lambda:dsrc.train.show_at(1), '2\n2')

In [ ]:
show_doc(DsrcSubset.decode_at)

<h4 id="<code>DsrcSubset.decode_at</code>" class="doc_header"><code>DsrcSubset.decode_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DsrcSubset.decode_at</code>(**`i`**, **\*\*`kwargs`**)

Decoded item at `i`

In [ ]:
test_eq(dsrc.valid.decode_at(1), [3,3])

In [ ]:
show_doc(DsrcSubset.decode)

<h4 id="<code>DsrcSubset.decode</code>" class="doc_header"><code>DsrcSubset.decode</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/05_data_source.ipynb#DataSource--" class="source_link" style="float:right">[source]</a></h4>

> <code>DsrcSubset.decode</code>(**`o`**, **\*\*`kwargs`**)

Transform decode

In [ ]:
t = dsrc.valid[1]
test_eq(dsrc.valid.decode(t), [3,3])

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_external.ipynb.
Converted 04_data_core.ipynb.
Converted 05_data_source.ipynb.
Converted 06_vision_core.ipynb.
Converted 07_pets_tutorial-meta.ipynb.
Converted 07_pets_tutorial.ipynb.
Converted 08_vision_augment.ipynb.
Converted 10_layers.ipynb.
Converted 11_optimizer.ipynb.
Converted 12_learner.ipynb.
Converted 13_callback_schedule.ipynb.
Converted 14_callback_hook.ipynb.
Converted 15_callback_progress.ipynb.
Converted 16_callback_tracker.ipynb.
Converted 17_callback_fp16.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_learner.ipynb.
